In [79]:
from z3 import *
BoolRef.__and__ = lambda self, rhs: And(self,rhs)
BoolRef.__or__ = lambda self, rhs: Or(self,rhs)
BoolRef.__xor__ = lambda self, rhs: Xor(self,rhs)
BoolRef.__invert__ = lambda self: Not(self)
#BoolRef.__rshift__ = lambda self, rhs: Implies(self,rhs)
BoolRef.__gt__ = lambda self, rhs: Implies(self,rhs) # better precednece. Bad gecause greater than is actually oppositive meaning
# implies is that conclusion is a <= b if a is 0, no conclusion. if a is 1 then b has to be 1 
# so there is a reasonalbe sense in wich a <= b is a implies b. ugh
# maybe > is better in the sense that it makes not so much sense to use strict comparison of booleans




In [251]:
# https://pypi.org/project/pyrsistent/
# python ADTs
x, y,a,b,c,z =Reals('x y a b c z')
(x + y).num_args()

(x + y).decl()
(x + y).children()
(x + y).children()[0]
(x + y).decl() == (x + y).decl() 

True

In [231]:
'''
It isn't clear that we want to use pure syntactic unifica

After we get it, make sure it is SAT (non contradictory)

for example if x == (y + 1) in one location and 1 + y in another, I think it'll still work
That's kind of nice
What about (x + 1) == (y + 1 + 2) ? Ok. I guess that is kind of hard.
I guess we could be querying z3 as we go down. No, just do what you can.
    (all learned eqaualitys)  => (t1 => t2) comes back unsat, we're good
    We don't need equivlance, we need implication which could be weaker.
If they are structurally identical, we good. But if they aren't perhaps you learn enough elsewhere

what about.
(x + 1) == (1 + y)
it would perhaps be nice if this could learn x == y


We're trying to synthesize the necessary equality constraints. From that perspective

We could use the countermodel to avoid having to ask quite so often.

if the operation is commutative, we may want to try it that way.
We may also want to flip into sympy for some things. I think sympy has matching/unification capabilities that work modulo commutativity.


a timeout stopped search
'''
import itertools
def unify(e1,e2):
    if is_const(e1):
        return [e1 == e2]
    else:
        if len(e1.children()) != len(e2.children()) or e1.decl() != e2.decl():
            return [e1 == e2] #[BoolVal(False)] # I guess we don't need to return False, since we have to check anyhow
        else:
            return itertools.chain.from_iterable([ unify(c1,c2) for c1,c2 in zip( e1.children() , e2.children()) ] )

In [ ]:
def apply(goal, th):
    new_goals = []
    if is_quantifier(th):
        th = th.body()
    while is_impl(th):
        new_goals.append(th.children[0])
        th = th.children[1]
    pred = unify(th,goal) 
    prove( Implies( pred , Implies(th.body(), goal)))
    return pred, new_goals
    

In [246]:
list(unify( x + (x + x), y+(y+y) ))

[x == y, False]

In [255]:
t = Tactic("solve-eqs")
t(And(*unify( (x + y) + z ,  (a + b) + c  )))

[[]]

In [ ]:
def prove2(claim):
        s = Solver()
        s.add(Not(claim))
        r = s.check()
        if r == unsat:
            return True
        if r  == sat:
            print("Countermodel : " + str(s.model()))
            return False
        return False
    
class CoreGoal():
    def __init__(self, th, ctx=[]): # context may include necessary theorems and definitions
        goal = th
        self.th = th # store original statement of theorem
        self.goals = [(ctx,goal)] #proof state
        self.proven = False
    def set_state(self, goals2):
        cond = Or([Implies(And(*ctx), goal) for ctx,goal in goals2])
        ctx, goal = self.goals[-1]
        if prove2(Implies(cond, Implies(ctx, goal)):
            self.goals.pop()
            self.goals.append(goals2)
            return True
        else:
            return False
    def smt(self):
        ctx, goal = self.goals[-1]
        if prove2( Implies(And(*ctx), goal)):
           self.goals.pop()
           return True
        else:
           return False
    def get_state(self):
        return self.goals[-1] #deepcopy(self.goals[-1])
    def qed(self):
        if len(self.goals) == 0:
            self.proven == True
            return True
        else:
            return False
    '''def axiom(self, ax):
       ctx, goal = self.goals[-1] 
       ctx.append(p.ax)
       return True
    '''
    def append_context(self, p):
       if p.proven:
          ctx, goal = self.goals[-1] 
          ctx.append(p.th)
          return True
       else:
          return False
        
class Axiom(CoreGoal):
      def __init__(self, th)
             self.th = th
             self.proven = True
             self.goals = []
     
def induction(t, p): #givne data type, return simplex induction principle for that data type
    return Axiom(induction(t, p)) # p is proposition to instantiate axiom with
        

In [4]:
#schema/templates for polymorphic tuples and sums. 
def Tuple(a,b):
    Type = Datatype('Tuple(f{(a.name(),b.name())})')
    # Constructor cons: (Int, List) -> List
    Type.declare('pair', ('fst', a), ('snd', b))
    # Create the datatype
    Type = Type.create()
    #print is_sort(List)
    return Type #, (Type.pair, Type.fst, Type.snd) #{t: Type, pair:Type.pair, fst:Type.fst, }
def Either(a,b):
    Type = Datatype('Either')
    # Constructor cons: (Int, List) -> List
    Type.declare('left', ('getLeft', a))
    Type.declare('right', ('getRight', b))
    # Create the datatype
    Type = Type.create()
    #print is_sort(List)
    return Type #, (Type.left, Type.getLeft), (Type.right, Type.getRight)
#Oh TupleSort and DisjointSum are already things
#p, _, _ = TupleSort("pair",[IntSort(),BoolSort()])
#dir(p)
# hmm not in love with their conventions
# Unit
# SeqSort is built in list? 
# MaybeSort(a)
# Void?
# 

In [ ]:
# The complex numbers as a data type
# complex_thy should be a python module. 
complex_theory = []
axioms = []
p, _, _ = TupleSort("complex",[IntSort(),BoolSort()])
# Should they just be macro-ized functions or should they
# I think probably they should be made similar to bit vectors. There is a high level description and properties
# and then once you've simplified signifcatnly you can "bit-blast" by going to coordinates.
# BitVector and RealVector.
#cmul = Function("cmul", Sort("complex"), Sort("complex"))
class Z3Complex(DataTypeRef):
    def __mul__(self,rhs): 
    def __add__(self,rhs):
    def __sub__(self,rhs):
    def __div__(self,rhs):
    def real(): #actually these are probably already part of the Ref
    def imag():


        
        

In [179]:
def V2(a):
    #print(f"V2<{a.name()}>")
    Type = Datatype(f"V2({a.name()})")
    # Constructor cons: (Int, List) -> List
    Type.declare('mk_v2', ('x2', a), ('y2', a))
    # Create the datatype
    Type = Type.create()
    #print is_sort(List)
    return Type  #, (Type.left, Type.getLeft), (Type.right, Type.getRight)
# Hmm This is fairly untenable. It would be really nice to get the right mk_v2 automatically.
dir(V2(V2(RealSort())).mk_v2(V2(RealSort()).mk_v2(1,2),V2(RealSort()).mk_v2(3,4))

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'get_id',
 'hash',
 'num_args',
 'params',
 'sexpr',
 'sort',
 'sort_kind',
 'translate',
 'use_pp']

In [182]:
a, b = Bools("a b")
p = Implies(a,b) & a
t = Tactic("unit-propagate")
t(p)

Z3Exception: unknown tactic 'unit-propagate'

In [121]:

#def gctx is_implies()
dir(SeqSort(IntSort()))

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'as_ast',
 'ast',
 'basis',
 'cast',
 'ctx',
 'ctx_ref',
 'eq',
 'get_id',
 'hash',
 'is_string',
 'kind',
 'name',
 'sexpr',
 'subsort',
 'translate',
 'use_pp']

In [205]:
def lift1(f,x):
    return lambda *args: f(x(*args))

def lift2(op,l,r):
    return lambda *args: op(l(*args), r(*args))

# interp is useful for transferring expressions into numpy, sympy
# but also for program extraction

from functools import reduce
import operator as op
def interp(a, *args):
    if is_true(a):
        return lambda *args: True
    elif is_false(a):
        return lambda *args: False
    elif is_int_value(a):
        return lambda *args: a.as_long()
    elif is_rational_value(a):
        n = a.numerator_as_long()
        d = a.denominator_as_long()
        return lambda *args: n / d
    #elif is_algebraic_value(a):
    #    pass
    elif is_const(a): # is free variable
        loc = [ind for ind, b in enumerate(args) if eq(a,b)]
        if len(loc) == 0:
            return a
        else:
            ind = loc[0]
            return lambda *args2: args2[ind]   
    b = [interp(c, *args) for c in a.children()]
    if is_and(a):
        return lambda *args: reduce(op.and_, [f(*args) for f in b])
    elif is_or(a):
        return lambda *args: reduce(op.or_, [f(*args) for f in b])
    elif is_app_of(a, Z3_OP_XOR):
        return lambda *args: reduce(op.xor, [f(*args) for f in b])
    elif is_add(a):
        return lambda *args: reduce(op.add, [f(*args) for f in b])
    elif is_mul(a):
        return lambda *args: reduce(op.mul, [f(*args) for f in b])
    elif len(b) == 1:
        n = b[0]    
        if is_not(a):
            return lift1(op.invert , n)# lambda *args: ~n
    elif len(b) == 2:
        l,r = b
        if is_eq(a):
            return lift2(op.eq, l,r) #lambda *args: l == r
        elif is_distinct(a): # This can be multi_argument
            return lift2(op.ne, l,r) # lambda *args: l != r
        elif is_sub(a):
            return lift2(op.sub, l,r) # lambda *args: l - r
        elif is_app_of(a, Z3_OP_POWER):
            return lift2(op.pow, l,r) #lambda *args: l ** r
        elif is_div(a):
            return  lift2(op.truediv, l,r)# lambda *args: l / r
        elif is_idiv(a):
            return lift2(op.floordiv, l,r) # lambda *args: l // r
        elif is_mod(a):
            return lift2(op.mod, l,r) # lambda *args: l % r
        elif is_le(a):
            return lift2(op.le, l,r) # lambda *args: l <= r
        elif is_lt(a):
            return lift2(op.lt, l,r) # lambda *args: l < r
        elif is_ge(a):
            return lift2(op.ge, l,r) #lambda *args: l > r
        elif is_gt(a):
            return lift2(op.gt, l,r) # lambda *args: l >= r
        elif is_implies(a):
            return lambda *args: (~ l(*args) ) & r(*args) 
    print("unrecognized constructor: ", type(a))
    assert(False)


In [200]:
a = Bool('a')
interp(a)

x, y = Reals('x y')
interp(x + y + y + y * x) # Z3_OP_POWER
is_distinct(a != a)
#True & True
interp(1 + x + x ** 2)
eq(x,x)
interp(BoolVal(False))()
interp(x+y,x,y)(2,1)


3

In [267]:
y = Real("y")
f = Function("f", RealSort(), RealSort())
g = Function("g", RealSort(),BoolSort())
h = Function("h", IntSort(),BoolSort())
s = Solver()
s.add(ForAll([y], (f(y) == 7) == (y == 7)))
s.check()
s.model()

[f = [7 -> 7, else -> 1]]

In [212]:
a = Bool('a')
interp(Xor(a & a | a, a), a)(True)

[0]
[0]
[0]
[0]


False

In [213]:
x, y = Reals('x y')
interp(x + y + y + y * x, x ,y)(3,2)

[0]
[1]
[1]
[1]
[0]


13

In [210]:
eq(a,a)

True

In [144]:
x,y = Ints("x y")
f = Function("f", IntSort(), BoolSort())
g = Function("g", IntSort(), IntSort())
prove( Implies(  ForAll([x], Implies( Implies(f(g(y)), f(x)) , f(x) ))  , f(g(y)) ))
Implies(  ForAll([x], Implies( Implies(f(g(y)), f(x)) , f(x) ))  , f(g(y)) )

proved


Implies(ForAll(x, Implies(Implies(f(g(y)), f(x)), f(x))),
        f(g(y)))

In [ ]:
x,y = Ints("x y")
f = Function("f", IntSort(), BoolSort())
prove( Implies(  ForAll([x], Implies( Implies(f(y), f(x)) , f(x) ))  , f(y) ))

In [146]:
x,y = Ints("x y") #shouldn't prove
f = Function("f", IntSort(), BoolSort())
g = Function("g", IntSort(), BoolSort())
prove( Implies(  ForAll([x], Implies( Implies(g(y), f(x)) , f(x) ))  , g(y) ))
Implies(  ForAll([x], Implies( Implies(g(y), f(x)) , f(x) ))  , g(y) )

counterexample
[y = 1, f = [else -> True], g = [else -> False]]


Implies(ForAll(x, Implies(Implies(g(y), f(x)), f(x))), g(y))

In [199]:
dir(z3core)

['ApplyResultObj',
 'Ast',
 'AstMapObj',
 'AstVectorObj',
 'Config',
 'Constructor',
 'ConstructorList',
 'ContextObj',
 'Elementaries',
 'FixedpointObj',
 'FuncDecl',
 'FuncEntryObj',
 'FuncInterpObj',
 'GoalObj',
 'Literals',
 'Model',
 'ModelObj',
 'OptimizeObj',
 'ParamDescrs',
 'Params',
 'Pattern',
 'ProbeObj',
 'RCFNumObj',
 'SolverObj',
 'Sort',
 'StatsObj',
 'Symbol',
 'TacticObj',
 'Z3Exception',
 'Z3_APP_AST',
 'Z3_ARRAY_SORT',
 'Z3_BOOL_SORT',
 'Z3_BV_SORT',
 'Z3_DATATYPE_SORT',
 'Z3_DEC_REF_ERROR',
 'Z3_EXCEPTION',
 'Z3_FILE_ACCESS_ERROR',
 'Z3_FINITE_DOMAIN_SORT',
 'Z3_FLOATING_POINT_SORT',
 'Z3_FUNC_DECL_AST',
 'Z3_GOAL_OVER',
 'Z3_GOAL_PRECISE',
 'Z3_GOAL_UNDER',
 'Z3_GOAL_UNDER_OVER',
 'Z3_INTERNAL_FATAL',
 'Z3_INT_SORT',
 'Z3_INT_SYMBOL',
 'Z3_INVALID_ARG',
 'Z3_INVALID_PATTERN',
 'Z3_INVALID_USAGE',
 'Z3_IOB',
 'Z3_L_FALSE',
 'Z3_L_TRUE',
 'Z3_L_UNDEF',
 'Z3_MEMOUT_FAIL',
 'Z3_NO_PARSER',
 'Z3_NUMERAL_AST',
 'Z3_OK',
 'Z3_OP_ADD',
 'Z3_OP_AGNUM',
 'Z3_OP_AND',
 'Z3_O

In [233]:
s = Solver()
a = Bool('a')
#s.add(ForAll([a], a == True))
#print(help(s.set))
#help(s)
set_param(proof=True)
s.add(a)
s.set(auto_config=False, mbqi = False, proof=True)
print(s.check())
print(s.unsat_core())
print(s.proof())
dir(s)



sat
[]


Z3Exception: b'there is no current proof'

In [240]:
#https://stackoverflow.com/questions/29577754/getting-proof-from-z3py
set_param(proof=True)
ctx = Context()
s = Solver(ctx=ctx)
x = Int('x', ctx=ctx)
s.add(x > 0)
s.add(x == 0)
s.check()
s.proof()

mp(mp(asserted(x > 0),
      rewrite((x > 0) == Not(x <= 0)),
      Not(x <= 0)),
   trans(monotonicity(trans(monotonicity(asserted(x == 0),
                                        (x <= 0) == (0 <= 0)),
                            rewrite((0 <= 0) == True),
                            (x <= 0) == True),
                      Not(x <= 0) == Not(True)),
         rewrite(Not(True) == False),
         Not(x <= 0) == False),
   False)

In [ ]:
set_param(proof=True)
ctx = Context()
s = Solver(ctx=ctx)
x = Int('x', ctx=ctx)
s.add(x > 0)
s.add(x == 0)
s.check()
s.proof()

In [248]:
 def Function2(name, *sig, ctx=ctx):
     """Create a new Z3 uninterpreted function with the given sorts.
 
     >>> f = Function('f', IntSort(), IntSort())
     >>> f(f(0))
     f(f(0))
     """
     sig = _get_args(sig)
     if z3_debug():
         _z3_assert(len(sig) > 0, "At least two arguments expected")
     arity = len(sig) - 1
     rng   = sig[arity]
     if z3_debug():
         _z3_assert(is_sort(rng), "Z3 sort expected")
     dom   = (Sort * arity)()
     for i in range(arity):
         if z3_debug():
             _z3_assert(is_sort(sig[i]), "Z3 sort expected")
         dom[i] = sig[i].ast
     #ctx = rng.ctx
     return FuncDeclRef(Z3_mk_func_decl(ctx.ref(), to_symbol(name, ctx), arity, dom, rng.ast), ctx)

In [273]:
set_param(proof=True)
ctx = Context()
s = Solver(ctx=ctx)
s.set(auto_config=False, mbqi = False)
x = Int('x', ctx=ctx)
y = Int('y', ctx=ctx)
f = Function("f", IntSort(ctx=ctx), BoolSort(ctx=ctx))
s.add(ForAll([x],f(x) == True, patterns=[f(x)], weight=0))
s.add(Not(f(y) == True))
s.check()
s.proof()


unit-resolution(quant-inst(Or(Not(ForAll(x, f(x))), f(y))),
                mp~(mp(mp(asserted(ForAll(x, f(x) == True)),
                          quant-intro(proof-bind(Lambda(x,
                                        rewrite((f(x) ==
                                        True) ==
                                        f(x)))),
                                      (ForAll(x,
                                        f(x) == True)) ==
                                      (ForAll(x, f(x)))),
                          ForAll(x, f(x))),
                       rewrite((ForAll(x, f(x))) ==
                               (ForAll(x, f(x)))),
                       ForAll(x, f(x))),
                    nnf-pos(proof-bind(Lambda(x,
                                        refl(~(f(x), f(x))))),
                            ~(ForAll(x, f(x)),
                              ForAll(x, f(x)))),
                    ForAll(x, f(x))),
                mp(mp(asserted(Not(f(y) == True)),
                      monotonicity(rewrite((f(y) == True) ==
                                        f(y)),
                                   Not(f(y) == True) ==
                                   Not(f(y))),
                      Not(f(y))),
                   rewrite(Not(f(y)) == Not(f(y))),
                   Not(f(y))),
                False)

In [8]:
x = Int('x')
y = Int('y')
f = Function("f", IntSort(), BoolSort())
p = Implies(ForAll([x],f(x) == True, patterns=[f(x)]), f(y) == True)
t = Tactic("horn")
t(p)

Z3Exception: b'recursive predicate (P!!3 (:var 0)) occurs nested in the body of a rule'

Ok. What if I just did a LITTLE bit of the unification.
Instantiate with fresh variables,
traverse down through b(x) and c in lock step. and add a constraint everytime we hit a variable

Reflect the z3 ast into itself as an ADT. That would be funny.
enum(Op_type)
ModelEval( z3_expr , m)

In [ ]:
unit-resolution(quant-inst(¬(∀x : f(x)) ∨ f(y)), mp~(mp(mp(asserted(∀x : f(x) = True), quant-intro(proof-bind(∃x : rewrite((f(x) = True) = f(x))), (∀x : f(x) = True) = (∀x : f(x))), ∀x : f(x)), rewrite((∀x : f(x)) = (∀x : f(x))), ∀x : f(x)), nnf-pos(proof-bind(∃x : refl(~(f(x), f(x)))), ~(∀x : f(x), ∀x : f(x))), ∀x : f(x)), mp(mp(asserted(¬(f(y) = True)), monotonicity(rewrite((f(y) = True) = f(y)), ¬(f(y) = True) = ¬f(y)), ¬f(y)), rewrite(¬f(y) = ¬f(y)), ¬f(y)), False)

In [260]:
dir(Pattern(f(x)))

['__bool__',
 '__class__',
 '__ctypes_from_outparam__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_as_parameter_',
 '_b_base_',
 '_b_needsfree_',
 '_objects',
 '_type_',
 'from_param',
 'value']

In [9]:
[ meth for meth in dir(z3core) if "implied" in meth ]

['Z3_get_implied_equalities']

In [11]:
help(Z3_get_implied_equalities)

Help on function Z3_get_implied_equalities in module z3.z3core:

Z3_get_implied_equalities(a0, a1, a2, a3, a4, _elems=<z3.z3core.Elementaries object at 0x104585b38>)



In [24]:
s = Solver()
x,y = Reals("x y")
s.add(Implies(x + 1 == 2, y + 1 == 2))
help(s.consequences)
#s.check()
s.consequences([x], [y,x])

Help on method consequences in module z3.z3:

consequences(assumptions, variables) method of z3.z3.Solver instance
    Determine fixed values for the variables based on the solver state and assumptions.
    >>> s = Solver()
    >>> a, b, c, d = Bools('a b c d')
    >>> s.add(Implies(a,b), Implies(b, c))
    >>> s.consequences([a],[b,c,d])
    (sat, [Implies(a, b), Implies(a, c)])
    >>> s.consequences([Not(c),d],[a,b,c,d])
    (sat, [Implies(d, d), Implies(Not(c), Not(c)), Implies(Not(c), Not(b)), Implies(Not(c), Not(a))])



(unknown, [])

In [ ]:
 def trail_levels(self):
     """Return trail and decision levels of the solver state after a check() call.
     """
     trail = self.trail()
     levels = (ctypes.c_uint * len(trail))()
     Z3_solver_get_levels(self.ctx.ref(), self.solver, trail.vector, len(trail), levels)
     return trail, levels
 def consequences(self, assumptions, variables):
     """Determine fixed values for the variables based on the solver state and assumptions.
     >>> s = Solver()
     >>> a, b, c, d = Bools('a b c d')
     >>> s.add(Implies(a,b), Implies(b, c))
     >>> s.consequences([a],[b,c,d])
     (sat, [Implies(a, b), Implies(a, c)])
     >>> s.consequences([Not(c),d],[a,b,c,d])
     (sat, [Implies(d, d), Implies(Not(c), Not(c)), Implies(Not(c), Not(b)), Implies(Not(c), Not(a))])
     """
     if isinstance(assumptions, list):
         _asms = AstVector(None, self.ctx)
         for a in assumptions:
             _asms.push(a)
         assumptions = _asms
     if isinstance(variables, list):
         _vars = AstVector(None, self.ctx)
         for a in variables:
             _vars.push(a)
         variables = _vars
     _z3_assert(isinstance(assumptions, AstVector), "ast vector expected")
     _z3_assert(isinstance(variables, AstVector), "ast vector expected")
     consequences = AstVector(None, self.ctx)
     r = Z3_solver_get_consequences(self.ctx.ref(), self.solver, assumptions.vector, variables.vector, consequences.vector)
     sz = len(consequences)
     consequences = [ consequences[i] for i in range(sz) ]
     return CheckSatResult(r), consequences
        
    assumptions = _get_args(assumptions)
         num = len(assumptions)
         _assumptions = (Ast * num)()
         for i in range(num):
             _assumptions[i] = assumptions[i].as_ast()
         r = Z3_solver_check_assumptions(self.ctx.ref(), self.solver, num, _assumptions)
         return CheckSatResult(r)

    
Z3_get_implied_equalities (Z3_context c, Z3_solver s, unsigned num_terms, Z3_ast const terms[], unsigned class_ids[])

Z3_lbool Z3_API 	Z3_solver_get_consequences (Z3_context c, Z3_solver s, Z3_ast_vector assumptions, Z3_ast_vector variables, Z3_ast_vector consequences)

In [27]:
def implied_equalities(self,terms):
    num = len(terms)
    _terms = (Ast * num)()
    classes = (ctypes.c_uint * num)()
    for i in range(num):
        _terms[i] = terms[i].as_ast()
    r = Z3_get_implied_equalities(self.ctx.ref(), self.solver, num, _terms, classes)
    print(r)
    return classes
Solver.implied_equalities = implied_equalities

In [44]:
s = Solver()
x,y,z = Reals("x y z")
f = Function("f", RealSort(), BoolSort())
s.add( ForAll([z], Implies(f(z) == f(y), z == y) )) # a kind of injectivity.
s.add( f(x+1) == f(y+1) )
#s.add(x == y, y == z)
help(s.consequences)
#s.check()
terms = [x,y]
res = s.implied_equalities(terms)
for i in range(len(terms)):
    print(res[i])

Help on method consequences in module z3.z3:

consequences(assumptions, variables) method of z3.z3.Solver instance
    Determine fixed values for the variables based on the solver state and assumptions.
    >>> s = Solver()
    >>> a, b, c, d = Bools('a b c d')
    >>> s.add(Implies(a,b), Implies(b, c))
    >>> s.consequences([a],[b,c,d])
    (sat, [Implies(a, b), Implies(a, c)])
    >>> s.consequences([Not(c),d],[a,b,c,d])
    (sat, [Implies(d, d), Implies(Not(c), Not(c)), Implies(Not(c), Not(b)), Implies(Not(c), Not(a))])

1
0
1


In [45]:
s = Solver()
x,y,z = Reals("x y z")
f = Function("f", RealSort(), BoolSort())
#s.add( ForAll([z], Implies(f(z) == f(y), z == y) )) # a kind of injectivity.
s.add( f(x) )
s.add( f(y) )
#s.add(x == y,' 'y == z)
#s.check()
terms = [x,y]
res = s.implied_equalities(terms)
for i in range(len(terms)):
    print(res[i])

1
0
1


In [41]:
s = Solver()
x,y,z = Reals("x y z")
f = Function("f", RealSort(), BoolSort())
s.add( ForAll([z], Implies(  (0 <= z) == (0 <= y), z == y) )) # a kind of injectivity.
s.add( (0 <= x) == (1 <= y) )
#s.add(x == y, y == z)
help(s.consequences)
#s.check()
terms = [x,y]
res = s.implied_equalities(terms)
for i in range(len(terms)):
    print(res[i])

Help on method consequences in module z3.z3:

consequences(assumptions, variables) method of z3.z3.Solver instance
    Determine fixed values for the variables based on the solver state and assumptions.
    >>> s = Solver()
    >>> a, b, c, d = Bools('a b c d')
    >>> s.add(Implies(a,b), Implies(b, c))
    >>> s.consequences([a],[b,c,d])
    (sat, [Implies(a, b), Implies(a, c)])
    >>> s.consequences([Not(c),d],[a,b,c,d])
    (sat, [Implies(d, d), Implies(Not(c), Not(c)), Implies(Not(c), Not(b)), Implies(Not(c), Not(a))])

-1
0
0


In [251]:
help(BoolSort)

Help on function BoolSort in module z3.z3:

BoolSort(ctx=None)
    Return the Boolean Z3 sort. If `ctx=None`, then the global context is used.
    
    >>> BoolSort()
    Bool
    >>> p = Const('p', BoolSort())
    >>> is_bool(p)
    True
    >>> r = Function('r', IntSort(), IntSort(), BoolSort())
    >>> r(0, 1)
    r(0, 1)
    >>> is_bool(r(0, 1))
    True



In [247]:
help(Int)

Help on function Int in module z3.z3:

Int(name, ctx=None)
    Return an integer constant named `name`. If `ctx=None`, then the global context is used.
    
    >>> x = Int('x')
    >>> is_int(x)
    True
    >>> is_int(x + 1)
    True



In [221]:
help(set_param)

Help on function set_param in module z3.z3:

set_param(*args, **kws)
    Set Z3 global (or module) parameters.
    
    >>> set_param(precision=10)



In [231]:
set_option("produce-proofs",True)

In [29]:
def instantiate(f, *args):
    if is_quantifier(f):
        if f.is_forall() or f.is_lambda():  
            return substitute_vars(f.body(), *args)
x = Real("x") 
y = Real("y") 
#print(ForAll([x], x == 8).body())
#instantiate(ForAll([x,y ], x == 8 + y), x, y) 
#dir(ForAll([x,y ], x == 8 + y))
dir(Tuple(IntSort(),IntSort()))

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'accessor',
 'as_ast',
 'ast',
 'cast',
 'constructor',
 'ctx',
 'ctx_ref',
 'eq',
 'fst',
 'get_id',
 'hash',
 'is_pair',
 'kind',
 'name',
 'num_constructors',
 'pair',
 'recognizer',
 'sexpr',
 'snd',
 'subsort',
 'translate',
 'use_pp']

In [19]:
def inductionNat(self):
    assert(self.num_vars() == 1 and self.var_sort(0) == IntSort() and self.is_forall())
    n = FreshInt()
    return instantiate(self, IntVal(0)) & ForAll([n],instantiate(self, n) & (n > 0) > instantiate(self, n+1))
# This is another style. Just attach the thing to     
QuantifierRef.inductionNat = inductionNat
# We are not doing termination metrics?
# General purpose inductive data type induction

In [ ]:
def goalctx(p):
    if is_implies(p):
        hyp, goal = p.children()
        return hyp, goal
# do I really need a goal stack?

def and_list(p):
    if is_and(p):
        return concatmap( and_list, p.children()
        #return and_list() + and_list()
    else:
        return [p]
                         

In [60]:
And(a,a,a).children()

[a, a, a]

In [ ]:
def induction(self):
    assert(is_quantifier(self) and self.is_forall() and self.num_vars() == 1) #we can eventually relax vars = 1
    datatype = self.var_sort()
    nc = datatype.num_constructors()
    cs = [datatype.constructor(i) for i in range(nc)]
    reduce(And, [   ])
    cts = [c.sort() for c in cs]
    acc = True
    for c in cs:
        Fresh
    terms = [ freshies = ForAll(freshies, instantiate(self, c( *freshies) ))  for c in cs]
    return reduce(bv_and, terms)
    #recognizer(0) good for match
#QuantifierRef.induction = induction    
#destruct

In [25]:
def intros(self): #name = hint maybe later
    assert(self.is_forall())
    vs = [FreshConst(self.var_sort(i) , prefix=self.var_name(i)) for i in range(self.num_vars())]
    return vs ,instantiate(self,*vs) # possibly return vs also? Then python context is kind of proof context
    '''if_implies(self):
        
    '''
    
#QuantifierRef.intros = intros

In [193]:
a = Bool("a")
#3*a + 4*a <= 7
Int(a)

AttributeError: 'BoolRef' object has no attribute 'is_real'

In [ ]:
#put valid step at the end of every proof change?
def valid_step(ctx1, g1, ctx2, g2):
    claim1 = Implies(And(*ctx1),g1)
    claim2 = Implies(And(*ctx2),g2)
    prove(Implies(claim1, claim1)
    

In [126]:
def prove2(claim):
        s = Solver()
        s.add(Not(claim))
        r = s.check()
        if r == unsat:
            return True
        if r  == sat:
            print("Countermodel : " + str(s.model()))
            return False
        return False

In [137]:
class Proof():
    def __init__(self, goal, name=None): # Taken a name for the theorem?
        self.goals = [([],goal)]
        self.proven = False
        self.name = name
    #def intros(self): #intro_all        
    #    self.goals.append( (ctx, goal.intros())  )
    #    return self
    def equiv(self, goal2):
        ctx, goal1 = self.goals.pop()
        if prove2(Implies(And(*ctx), goal1 == goal2)):
            g = goal2
        else:
            g = goal1
        self.goals.append( (ctx, g))
        return self
    def __eq__(self,rhs):
        return self.equiv(rhs)
    #def assert(): #put new goal in stack with current context. Put into context of 1 below top
    #def assume(): #just put crap in the context.
    def intro_all(self): #name = hint maybe later
        ctx, goal = self.goals.pop()
        assert(goal.is_forall())
        vs = [FreshConst(goal.var_sort(i) , prefix=goal.var_name(i)) for i in range(goal.num_vars())]
        g = instantiate(goal,*vs) 
        self.goals.append( (ctx + vs, g)) # wait. I should keep propositions and variables seperate
        return self
    def intro_imp(self): #intro_impl
        ctx, goal = self.goals.pop()
        if is_implies(goal):
            a, b = goal.children()
            ctx.append(a)
            self.goals.append((ctx,b))
        else:
            self.goals.append((ctx,goal))
        return self
    def split(self): #z3 tactic split-clauses?
        ctx, goal = self.goals.pop()
        if is_and(goal):
            for c in goal.children():
                self.goals.append((ctx,c))
        else:
            self.goals.append((ctx,goal))
        return self
    def z3_tactic(self,t):
        t = Tactic(t)
        ctx, goal = self.goals.pop()
        #g = t(Implies(And(*ctx), goal)).as_expr()
        g = t(goal).as_expr()
        self.goals.append(([],g))
        return self
    def simpl(self):
        return self.z3_tactic("simplify")
    def congruence(self):
        #maybe search for equalities. And put them in the goal
        return self.z3_tactic("solve-eqs")
    def smt(self):
        ctx, goal = self.goals.pop()
        s = Solver()
        #s.set(**keywords)
        claim = Implies(And(*ctx), goal)
        s.add(Not(claim))
        r = s.check()
        if r  == sat:
            print("Countermodel : " + str(s.model()))
        assert(r == unsat)
        return self
    def destruct(self):
        ctx, goal = self.goals.pop()
        if is_bool(goal):
            ctx1 = ctx.copy()
            ctx2 = ctx.copy()
            ctx1.append(goal == True)
            ctx2.append(goal == False)
            self.goals.append((ctx2, BoolVal(False) ))
            self.goals.append((ctx1, BoolVal(True) ))
        else:
            self.goals.append((ctx, goal))    
        return self
    def forget(self,n):
        ctx, goal = self.goals.pop()
        ctx.pop(n)
        self.goals.append((ctx, goal))  
        return self
    def qed(self):
        if len(self.goals) == 0:
            self.proven = True
            # add self to global proof context if self.name is not None
    def get_ctx(self,n):
        return self.goals[-1][0][n]
    def __str__(self):
        if len(self.goals) >= 1:
            ctx, goal = self.goals[-1]
            return "".join([f"[{i}] {str(c)} : {str(c.sort())}\n" for i, c in enumerate(ctx)]) + "----------------\n" + f"{str(goal)} : {str(goal.sort())}"
        else:
            return "No Goals Left"
    def __repr__(self):
        return str(self)

In [214]:
x = Real("x")
Proof(x**2 - 1 == 0).equiv((x+1)*(x-1) == 0).equiv((x == 1) | (x == -1))

----------------
Or(x == 1, x == -1) : Bool

In [141]:
x = Real("x")
Proof(x**2 - 1 == 0) == ((x-1)*(x+1) == 0) == ((x == 1) | (x == -1))
# I don't love this

----------------
Or(x == 1, x == -1) : Bool

In [171]:
a, b = Bools('a b')
p = Proof((a & b) > b)
p.intro_imp().destruct() 
   .smt() \
   .smt() \
.qed()


[0] And(a, b) : Bool
[1] b == True : Bool
----------------
True : Bool

In [198]:
x = Real("x")
prove((x >= 1) > (x**6 >= 1))

proved


In [179]:
x = Real('x')
p = Proof(ForAll([x], x + 3 >= 4))
p.simpl().intro_all()
#Proof(a & b > a).z3_tactic("split-clause")
#dir(p.simpl().goals[0][1].result)
#type(p.simpl().goals[0][1]

[0] x!12 : Real
----------------
x!12 >= 1 : Bool

In [101]:
prove(BoolVal(True))

proved


In [ ]:
# Yea. A visitor?
# inrpret should be a indetity if context holds itself.
# lambdify gives sympy an intepretation.
# Hm. Maybe we should give a lambdify?
# Give a list of variables instead of ctx
def lambdify_z3(z3_expr, vs):
    if is_symbol
        ind = find z3_expr in vs
        return lambda *vs: vs[ind]
    if is_plus(z3_expr):
        lambda *xs:  lambdify_z3(z3_expr.child(0), vs)(*xs) + lambdify_z3(z3_expr.child(1), vs)(*xs)
    if is_and():
        lambda *xs: &
    if is_mul()
# maybe I want to cps this.

def interpret(z3_expr, ctx):
    if is_symbol(z3_expr):
        lalookup(z3_expr, ctx)
    if is_lambda(z3_expr):
        tz3 = FreshConst()
        return lambda t: intepret(substitute_vars(z3_expr.body(),   , ctx ++ {tz3: t} )
        
    if is_le():
    if is_impl():
    if is_and():
                &
    if is_mul():
    if is_
                                
                                  
        

In [ ]:

class Proof2():
    def __init__(self):
        self.goal = 

In [76]:
def forall(sort, expr):
    t  = FreshConst(sort)
    return ForAll([t], expr(t))


In [70]:
dir(b)

['__and__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'get_id',
 'hash',
 'num_args',
 'params',
 'sexpr',
 'sort',
 'sort_kind',
 'translate',
 'use_pp']

In [74]:
b._repr_html_()
str(b)
repr(b)

'b'

In [77]:
p

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [51]:
n = Int('n')
p = ForAll([n], n == n + 1)
g = Goal()
g.add(p)
g.add(n == n)

g[0]
g[1]
dir(g)
g.size()
g.get(0)
len(g)
g.inconsistent
del g

False

In [28]:
n = Int('n')
p = ForAll([n], n < n + 1)
p.inductionNat()
p.intros()

n!9 < n!9 + 1

In [56]:
tib = Tuple(IntSort(),BoolSort())
simplify(tib.fst( tib.pair(3,True) ))

def dup(t):
    x = FreshConst(t)
    return Lambda([x], Tuple(t,t).pair(x,x))

def iden(t):
    x = FreshConst(t)
    return Lambda([x],x)

def match(x, **kwargs):
    dt = x.sort()
    nc = dt.num_construcotrs()
    nc 
    
    x.
    
'''def comp(a,b,c):
    f = FreshConst(FunctionSort(a,b))
    a = FreshConst(a)
    g = FreshConst(FunctionSort(b,c))
    return Lambda([f,g], Lambda([a], g(f(a))))
def comp(f,g):
    a = FreshConst(f.domain)
    return Lambda([a], g(f(a)))
    
'''
#Function


dup(IntSort())
dir(Function("f",IntSort(),IntSort()))
#comp(IntSort(),BoolSort(),RealSort())
#def par(f,g):
#    x = FreshConst(t)
#    return Lambda([x], Tuple(t,t).pair(x,x))



['__bool__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'arity',
 'as_ast',
 'as_func_decl',
 'ast',
 'ctx',
 'ctx_ref',
 'domain',
 'eq',
 'get_id',
 'hash',
 'kind',
 'name',
 'params',
 'range',
 'sexpr',
 'translate',
 'use_pp']

In [ ]:
'''
# Implementing "Haskell" in Z3Py
def match(x, *args):
    sort = x.sort()

a desguaring match
def is_empty(l): match(l, cons = lambda x,xs: True, nil=False)
def fix(("name", types), body): 



Unit = CreateSort("Unit")?
Void = ?

par = lambda f,g: 
dup = lambda x: pair(x,x)

dup = lambda t x: Tuple(t,t).pair(x,x) 
dup = lambda t : Lambda([x], Tuple(t,t).pair(x,x)) # huh

id = lambda t x: x
id2 = lambda t : x = FreshConst(t); Lambda([x], x)
id3 = 

Interconverting between the different forms. Possible?
Lambda => lambda  is substititue_vars.
lambda => Lambda  is  



Z = CreateSort("Z") ? Typelevel naturals?
'''

In [32]:
p, q = Bools("p q")
#prove( ((p > q) > q) > p)
prove( ((p > q) > p) > p) # peirce's law

proved


In [2]:
x = Int('x')
prove(ForAll([x],x >= 3))


counterexample
[]


In [32]:
p = ForAll([x],x >= 3)
is_quantifier(p)
dir(p)
p.is_forall()
p.is_exists()
dir(p)
p.sexpr()
p.body()
dir(p)
p.as_ast()
dir(p)
p.var_name(0)
p.num_vars()
dir(p)
dir(p)
p.children()

Var(0) >= 3

In [33]:
describe_tactics()

ackermannize_bv : A tactic for performing full Ackermannization on bv instances.
subpaving : tactic for testing subpaving module.
horn : apply tactic for horn clauses.
horn-simplify : simplify horn clauses.
nlsat : (try to) solve goal using a nonlinear arithmetic solver.
qfnra-nlsat : builtin strategy for solving QF_NRA problems using only nlsat.
nlqsat : apply a NL-QSAT solver.
qe-light : apply light-weight quantifier elimination.
qe-sat : check satisfiability of quantified formulas using quantifier elimination.
qe : apply quantifier elimination.
qsat : apply a QSAT solver.
qe2 : apply a QSAT based quantifier elimination.
qe_rec : apply a QSAT based quantifier elimination recursively.
psat : (try to) solve goal using a parallel SAT solver.
sat : (try to) solve goal using a SAT solver.
sat-preprocess : Apply SAT solver preprocessing procedures (bounded resolution, Boolean constant propagation, 2-SAT, subsumption, subsumption resolution).
ctx-solver-simplify : apply solver-based context

In [ ]:
 ParThen(Tactic('split-clause'), Tactic('propagate-values'))
    'nnf' 'simplify' 'skip' 'snf' 'qe-light' 'qe2' 'smt' 
    'elim-and' 'injectivity' 'fail' 'fail-if-undecided'
    'ctx-solver-simplify'

In [196]:
a, b = Bools("a b")
p = Implies(a,b) & a

#t = Tactic("propagate-values")

#t = Tactic("simplify")
#p = t(p).as_expr()
#t = Tactic("sat-preprocess")
t = Tactic("horn-simplify")
t(p)
#t = Tactic("unit-subsume-simplify") #didnt do anything


[[]]

Tactics.
Z3 has some 

In [37]:
t = Tactic("snf")
p = ForAll([x],x >= 3)
t(p)

[[ForAll(x, x >= 3)]]

In [45]:
t = Tactic("qe-light")
t(p)

[[False]]

In [46]:
t = Tactic("qe-light")
t(Exists([x], x >=3))

[[True]]

In [47]:
t = Tactic("snf")
t(Exists([x], x >=3))

[[x!5 >= 3]]

In [50]:
t = Tactic("snf")
p = Not(ForAll([x],x >= 3))
t(p) 

[[Not(x!8 >= 3)]]

In [41]:
t = Tactic("factor")
t(x**2 - 1 == 0)

[[Or(x + 1 == 0, x + -1 == 0)]]

In [71]:
x = Real('x')

#help(Goal)
g = Goal([Or(x == 7, x == 3)])
g

[]

In [3]:
fac = RecFunction('fac', IntSort(), IntSort())
n = Int('n')
RecAddDefinition(fac, n, If(n == 0, 1, n*fac(n-1)))

In [107]:
#l = List("l")
nil
cons

NameError: name 'nil' is not defined

In [4]:
s = Solver()
s.add(fac(n) == 6)
s.check()
s.model() #z3 can backwards solve functions.

[n = 3, fac = [0 -> 1, else -> fac(-1 + Var(0))*Var(0)]]

In [5]:
help(Forall)

NameError: name 'Forall' is not defined

In [ ]:
help(z3)

In [ ]:
help(z3)

In [ ]:
help(Forall)

In [54]:
a = Bool("a")
b = Bool("b")
a | b

TypeError: unsupported operand type(s) for |: 'BoolRef' and 'BoolRef'

In [25]:
a = Bool("a")
~a
a >> a
(a & a & a) >> a # precdnence is bad.
(a | a) >> a
~a & a > a

proved


In [102]:
1 <= 2  #Ugh. Idunno, Sympy uses >> as implies. That's not bad.It is scary to mistake with a bitvector

True

In [58]:
a & b #These seem like sensible operators to be in z3 for bools

And(a, b)

In [103]:
a <= b

a <= b

In [59]:
a | b

Or(a, b)

In [60]:
a & b & a & b

And(And(And(a, b), a), b)

In [65]:
7*b

If(b, 7, 0)

In [78]:
b = BitVec("b", 32)
a = BitVec("a", 32)

In [75]:
prove(b == b | b)

proved


In [76]:
prove(0 == b ^ b)

proved


In [79]:
prove(a == (a ^ b) ^ b)

proved


Proof by induction
It is questionable the degree to which z3 can handle proof by induction
Self note: A bounded size proof might be possible though? Use z3 as search procedure. Short circuit 
An application of a proof rule usually requires some unification and subsitution. z3 can do that.
Or what about just defining a GADT, which we kind of can because we can define ADT plus bizarro z3 conditions.

ADT("Vec n")



If we take z3 as a trusted base, could we build a python tactic system around it.
.auto() just calls z3
.intros() examine goal expression
.exists()
.
for each constructor
These are possibly gadts, or possibly considered as measures a la liquid haskell
c += [ForAll([n,a,v], (length v == n), patterns=[VCons(n,a,v)])]
c += [length VNil = 0]

alternatively
is_valid(d) = (Vec.n(d) == length(Vec.cdr(d))) & is_valid(Vec.cdr(d))

is_valid / smart_constructor pair

proof search by reflection of proof trees into z3.

traverse and map etc

def map(f): # over finite structures we can use templated map





Also agda style equality reasoning


Proof by induction can follow from using axiom scheme.

Imperative proofs. This may already be provided by Z3's Goal class?

contexts?

class Proof():
    __init__(self,goal, ctx=[]):
    self.context = ctx
    self.goal = goal
    def auto(self, hint_db = []):
        # do we want to give the context?
        prove(Implies(**ctx,goal))
        #or maybe we want to do an actual search
    def apply(self,proof):
        #proof.check() or
        if proof.is_proven:
            model = solve(proof.goal == self.goal)
        return self
    def simp():
        self.goal = simplify(self.goal)
    def intros(names?):
        if is_quantifier(goal) and is_forall(goal):
    def destruct(self):
    
    def induction(self):
        
    def reflexivity():
        prove(goal) 
        # or maybe auto
        
    def __next__(self) # use generator for proof stack?
    
I really need to start consolidating and writing

### The apply tactic. 

The apply tactic takes a theorem that contains as it's final implication something that is easily shown to imply the goal. You can then use this theorem and have new goals the correspond to the hypothesis of the theorem.
Easily shown usually works by unification. The syntax of the goals and the syntax of the result of the theorem have to match.


In [99]:
#Projective geometry

Point = DeclareSort('Point')
p, q = Consts('p q',Point)
collinear = Function("collinear", Point, Point, Point , BoolSort())
x = Consts('x', BoolSort())
solve(p == q)
Line = DeclareSort('Line')
p, q = Consts('p q',Line)

ForAll([p,q], collinear(p,q,p) == True)
ForAll([p,q], collinear(p,q,p) == True)

Exists([a,b,c], And(collinear(a,b,c), b != c , a != c))
ForAll([a,b,c,d], Implies( And(collinear(a,b,c) ,collinear(a,b,d)), collinear(b,d,c) ))

online(a,b,l), (c,d,l)=> oneline(a,c) online(d)
meetpoints



[q = Point!val!0, p = Point!val!0]


In [109]:
# Declare a List of integers
List = Datatype('List')
# Constructor cons: (Int, List) -> List
List.declare('cons', ('car', IntSort()), ('cdr', List))
# Constructor nil: List
List.declare('nil')
# Create the datatype
List = List.create()
#print is_sort(List)
cons = List.cons
car  = List.car
cdr  = List.cdr
nil  = List.nil


NameError: name 'Tuple' is not defined

In [ ]:
app = RecFunction('append', List, List, List)
l1 = Int('l1')
l2 = Int('l2')
RecAddDefinition(fac, n, If(l1 == nil, 1, n*fac(n-1)))

In [113]:
llen = RecFunction('length', List, IntSort())
l1 = Const('l1', List)
l2 = Int('l2')
RecAddDefinition(llen, l1, If(l1 == nil, 0, 1 + llen(cdr(l1))))

In [114]:
solve(llen(l1) == 4)

[l1 = cons(8, cons(7, cons(11, cons(10, nil)))),
 length = [nil -> 0, else -> 1 + length(cdr(Var(0)))]]


In [116]:
prove(Implies(l1 == nil, llen(l1) >= 0))

proved


In [119]:
l2 = Const('l2', List)
n = Int('n')
prove(Implies(And(l1 == cons(n,l2), llen(l2) >= 0), llen(l1) >= 0))

proved


In [ ]:
# This induction principle doesn't really make much sense.
Forall([p], Implies( [p(nil) , Implies(p(l1) , p(cons(n,l1))   ] , Forall([l], p(x))   )
                      

In [ ]:
induction( e  ):
    is_quantifier(e)
    if_forall(e)
    eval(body,nil)
    eval(body, l1)
    eval(body, cons(n,l1))

In [121]:
b = Bool('b')
Implies([b,b],b)

Z3Exception: True, False or Z3 Boolean expression expected. Received [b, b]

In [175]:
x = Real('x')
f = Lambda([x], x)
'''dir(f)
f.var_name(0)
dir(f)
f.num_vars()
f.var_sort(0)
if f.is_lambda:
    ss = [Var(i, f.var_sort(i)) for i in range(f.num_vars())]
    '''
substitute_vars(f.body(), x+8)
#f.body()
#f.sort()

x + 8

In [178]:
#v0 = Var(0, IntSort())
#v1 = Var(1, IntSort())
x  = Real('x')
#f  = Function('f', IntSort(), IntSort())
f = Lambda([x], Lambda([x], x))
# replace v0 with x+1 and v1 with x

substitute_vars(f.body(), x + 1)


Var(0)

In [ ]:
substitute
m.eval # in a model


In [184]:
dir(List)

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'accessor',
 'as_ast',
 'ast',
 'car',
 'cast',
 'cdr',
 'cons',
 'constructor',
 'ctx',
 'ctx_ref',
 'eq',
 'get_id',
 'hash',
 'is_cons',
 'is_nil',
 'kind',
 'name',
 'nil',
 'num_constructors',
 'recognizer',
 'sexpr',
 'subsort',
 'translate',
 'use_pp']

In [201]:

List.num_constructors()
List.accessor(0,0)
List.constructor(0)
List.constructor(1)
List.recognizer(0)
List.recognizer(1)


is(nil)

In [ ]:
[List.constructor(i) for i in range(d.num_constructors())
 # look for recusrive calls. Mutually recursive calls? Eeeh.

In [180]:
f = ForAll([x], x == 7)
f.is_forall()
def intros(f):
    if is_quantifier(f):
        if f.forall():            
            n = f.num_vars()
            da_vars = [Const(f.var_name(), f.var_sort(i)) for i in range(n)]
            substitute_vars(f.body(), *da_vars)


True

In [202]:
def Impland(hyps, res):
    return Implies(And(*hyps), res)

In [203]:
prove(Impland([x >= 3, x <= 4], x <= 10))

proved


In [ ]:
def ind_schema(p):
    n = FreshInt()
    Implies(And(p(0), Forall([n], Implies(p(n), p(n+1)) ), Forall([n]) p(n)) #Plus positivty of n
def ind_list(p):
    n = Fresh(List)
    Implies(And(p(List.nil), Forall([n], Implies(p(n), p(n+1))), Forall([n]) p(n)) #Plus positivty of n

https://arxiv.org/pdf/1905.05970.pdf - holpy

Instantiate.
Any proven theorem can be instantiated to particlaur values

Put z3 proofs in any inference rules?
assert(prove(a /\ b => b) == proved)
modus ponens (a,b,c |- d) (d' |- r) can be composed if d |- d' 
forward reonding
def sym:
   if is_eq:
      a,b = get left, get right
      if(prove(g >> b == a)
def forward(n):
   if prove(goal & ctx >> n):
       ctx += goal
       goal = n
tactic( in = None): # in makes the forward mode version
    
my_theorem = Proof("(forall x (x >= 7))").auto().
  


my_theroem

Tuple 

fst = 
snd = 


could add induction principle in as a triggered expression. Triggered on the particular theorem w are trying to prove it on.



Huh. Cody tells me he already did all this.
Do I believe him? Does it matter?
https://github.com/avigad/boole

Reflection of proof terms into z3 itself.
check = RecFunction(, Pf => Bool)
check = If(is_cons(), , )

cody is converned about apply tactic. 1. do we need it? 2. instantiate tactic

def instantiate(t, *args):
    subs(t,args)
    c = concl(t)
    hyps = hyps(t)
    if prove(goal == c):
        self.   
        self.goals += hyps
    
    
    subsitutie_vars(   )


def apply(t):
    # t.proven()
    if is_forall(t): #figure out how to instantiate quantified variables
        t2 = instantaitefresh(t)
        hyps, concs = impland(t2) # or maybe we don't have to instantiate to do this
        if prove( (forall concs) >> goal )): #Just prove the small step of the proof, which should be easier
            ctx += (self.goal == t2) #This is the unification predicate. Hmm. Do I need a forall here?
            goal.pop()
            goals += hyps
    else: # literally equal/ impoied
        hyp, conc = t
        if prove(conc >> goal) == proved:
            goal.pop()
            goal += hyp
def intros(): # suggested names? also could use [,,,] case patterns if we're being fancy
    is_forall:
        instantiatefresh(self.goal):


intros().auto() instatiates forall, then calls prove which has an implicit forall.


def induction(n):
    get_sort(n)
    goal = goals.pop()
    for constructor in
        ind_hyp = find recursive calls of type
        ctx = ctx.copy()
        goal += [(ctx, goal)]
    

Should we store the context and the goal seperately or just keep the z3 expression of the form Implies(yada,yada)


def qed():
    if prove(ctx >> goal):
        self.proven = True
    

(ctx >> goal)


__str__(self):
    if len(self.goals) >= 1:
        (ctx,goal) = self.goals[0]
        return concat([f"{str(c)} : {str(sort(c))}\n" for c in ctx]) + "----------------\n" + f"{str(goal)} : {str(sort(goal))"
    else:
        return "No Goals Left"
    
      
    
